In [ ]:
# OBJ3 Step 2 — Pre vs Post merge (robust paths + auto-download)
# Copy/paste this cell into your notebook and run.

from pathlib import Path
import pandas as pd
import urllib.request

HERE = Path.cwd() 

#  GitHub RAW URLs 
PRE_URL = "https://raw.githubusercontent.com/msusanrocha/DEGO_Project_Group03/main/data/quality/data_quality_report.csv"
POST_URL = "https://raw.githubusercontent.com/msusanrocha/DEGO_Project_Group03/main/data/quality/reports/post/data_quality_report_postclean.csv"
BEFORE_AFTER_URL = "https://raw.githubusercontent.com/msusanrocha/DEGO_Project_Group03/main/data/quality/before_after_comparison.csv"

#  local filenames if we need to download into the notebook folder 
PRE_LOCAL_FALLBACK = HERE / "data_quality_report.csv"
POST_LOCAL_FALLBACK = HERE / "data_quality_report_postclean.csv"
BEFORE_AFTER_LOCAL_FALLBACK = HERE / "before_after_comparison.csv"

def download_if_missing(url: str, dest: Path) -> Path:
    dest.parent.mkdir(parents=True, exist_ok=True)
    if dest.exists():
        return dest
    print(f"Downloading -> {dest.name}")
    urllib.request.urlretrieve(url, dest.as_posix())
    return dest

def find_upwards(rel_path: str) -> Path | None:
    """
    Search for a file by checking HERE and each parent directory.
    Example rel_path: "data/quality/data_quality_report.csv"
    """
    for base in [HERE] + list(HERE.parents):
        candidate = base / rel_path
        if candidate.exists():
            return candidate
    return None

def resolve_or_download(rel_path: str, url: str, fallback_dest: Path) -> Path:
    """
    1) Try to find file locally somewhere above current folder (repo root variations)
    2) If not found, download into the notebook folder (fallback_dest)
    """
    local = find_upwards(rel_path)
    if local is not None:
        return local
    return download_if_missing(url, fallback_dest)

# Resolve paths 
pre_path = resolve_or_download("data/quality/data_quality_report.csv", PRE_URL, PRE_LOCAL_FALLBACK)
post_path = resolve_or_download("data/quality/reports/post/data_quality_report_postclean.csv", POST_URL, POST_LOCAL_FALLBACK)

# before_after comparison: useful as a sanity check / executive table
before_after_path = resolve_or_download("data/quality/before_after_comparison.csv", BEFORE_AFTER_URL, BEFORE_AFTER_LOCAL_FALLBACK)

# Load PRE
pre = pd.read_csv(pre_path)

# Standardize column names 
pre.columns = [c.strip() for c in pre.columns]

required_pre_cols = {"stage", "issue_group", "rule_id", "field_path", "description", "count", "percent", "severity", "value_source"}
missing_pre = required_pre_cols - set(pre.columns)
if missing_pre:
    raise ValueError(f"PRE report missing columns: {sorted(missing_pre)}")

pre = pre[pre["stage"].astype(str).str.lower().eq("pre")].copy()

pre["count"] = pd.to_numeric(pre["count"], errors="coerce")
pre["percent"] = pd.to_numeric(pre["percent"], errors="coerce")

pre_keep = ["rule_id", "issue_group", "field_path", "description", "severity", "value_source", "count", "percent"]
pre = pre[pre_keep].rename(columns={"count": "pre_count", "percent": "pre_percent"})

# Load POST 
post = pd.read_csv(post_path)
post.columns = [c.strip() for c in post.columns]

if "issue_type" in post.columns and "issue_group" not in post.columns:
    post = post.rename(columns={"issue_type": "issue_group"})

required_post_cols = {"rule_id", "issue_group", "field_path", "description", "count", "percent", "severity"}
missing_post = required_post_cols - set(post.columns)
if missing_post:
    raise ValueError(f"POST report missing columns: {sorted(missing_post)}")

post["count"] = pd.to_numeric(post["count"], errors="coerce")
post["percent"] = pd.to_numeric(post["percent"], errors="coerce")

post_keep = ["rule_id", "issue_group", "field_path", "description", "severity", "count", "percent"]
# value_source exists in your post file; include it if present
if "value_source" in post.columns:
    post_keep.append("value_source")

post = post[post_keep].copy().rename(columns={"count": "post_count", "percent": "post_percent"})

# If both pre and post have value_source, keep pre's as "pre_value_source" and post's as "post_value_source"
if "value_source" in pre.columns and "value_source" in post.columns:
    pre = pre.rename(columns={"value_source": "pre_value_source"})
    post = post.rename(columns={"value_source": "post_value_source"})
elif "value_source" in pre.columns:
    pre = pre.rename(columns={"value_source": "pre_value_source"})
elif "value_source" in post.columns:
    post = post.rename(columns={"value_source": "post_value_source"})

#  Merge + deltas 
cmp = pre.merge(post, on="rule_id", how="outer", suffixes=("_pre", "_post"))

# Prefer post metadata when pre metadata is missing
for col in ["issue_group", "field_path", "description", "severity"]:
    pre_col = f"{col}_pre"
    post_col = f"{col}_post"
    if pre_col in cmp.columns and post_col in cmp.columns:
        cmp[col] = cmp[post_col].combine_first(cmp[pre_col])
        cmp = cmp.drop(columns=[pre_col, post_col])

cmp["pre_count"] = pd.to_numeric(cmp.get("pre_count"), errors="coerce")
cmp["post_count"] = pd.to_numeric(cmp.get("post_count"), errors="coerce")
cmp["pre_percent"] = pd.to_numeric(cmp.get("pre_percent"), errors="coerce")
cmp["post_percent"] = pd.to_numeric(cmp.get("post_percent"), errors="coerce")

cmp["delta_count"] = cmp["post_count"] - cmp["pre_count"]
cmp["delta_pp"] = cmp["post_percent"] - cmp["pre_percent"]  

#  Save output 
out_dir = HERE / "reports" / "evidence"
out_dir.mkdir(parents=True, exist_ok=True)

out_path = out_dir / "pre_post_comparison.csv"
cmp.to_csv(out_path, index=False)

# Quick sanity checks 
check_rules = ["R_APP_006", "R_APP_008", "R_APP_012", "R_APP_013", "R_APP_014", "R_APP_001", "R_APP_009"]
print("\nSanity check (selected rules):")
print(
    cmp[cmp["rule_id"].isin(check_rules)][
        ["rule_id", "severity", "pre_count", "pre_percent", "post_count", "post_percent", "delta_pp"]
    ].sort_values("rule_id")
)

# Top improvements 
print("\nTop improvements (delta_pp ascending):")
top_improvements = cmp.sort_values("delta_pp").head(15)
print(top_improvements[["rule_id", "severity", "pre_percent", "post_percent", "delta_pp", "pre_count", "post_count"]])

# Residual issues 
print("\nResidual issues (post_percent > 0):")
residual = cmp[(cmp["post_percent"].fillna(0) > 0)].copy()
severity_order = {"high": 0, "medium": 1, "low": 2}
residual["severity_rank"] = residual["severity"].astype(str).str.lower().map(severity_order).fillna(9)
residual = residual.sort_values(["severity_rank", "post_percent"], ascending=[True, False]).head(20)
print(residual[["rule_id", "severity", "post_percent", "post_count", "description"]])

# Residual organized --> severity
residual = cmp[(cmp["post_percent"].fillna(0) > 0)].copy()
severity_order = {"high": 0, "medium": 1, "low": 2}
residual["severity_rank"] = residual["severity"].astype(str).str.lower().map(severity_order).fillna(9)
residual = residual.sort_values(["severity_rank", "post_percent"], ascending=[True, False]).head(15)

residual[["rule_id", "severity", "post_percent", "post_count", "description"]]


Sanity check (selected rules):
      rule_id severity  pre_count  pre_percent  post_count  post_percent  \
0   R_APP_001     high      440.0        87.65         440         87.65   
5   R_APP_006      low      111.0        22.11           0          0.00   
7   R_APP_008      low      157.0        31.27           0          0.00   
8   R_APP_009   medium       39.0         7.77          39          7.77   
11  R_APP_012     high        2.0         0.40           0          0.00   
12  R_APP_013     high        1.0         0.20           0          0.00   
13  R_APP_014     high        1.0         0.20           0          0.00   

    delta_pp  
0       0.00  
5     -22.11  
7     -31.27  
8       0.00  
11     -0.40  
12     -0.20  
13     -0.20  

Top improvements (delta_pp ascending):
      rule_id severity  pre_percent  post_percent  delta_pp  pre_count  \
7   R_APP_008      low        31.27          0.00    -31.27      157.0   
5   R_APP_006      low        22.11          0.00  

,rule_id,severity,post_percent,post_count,description
0,R_APP_001,high,87.65,440,Missing or blank processing timestamp.
1,R_APP_002,high,1.59,8,One or more required applicant fields missing ...
21,R_DUP_003,high,1.20,6,Rows where SSN repeats across one or more reco...
2,R_APP_003,high,1.00,5,Both SSN and IP address missing/blank.
19,R_DUP_001,high,0.80,4,Rows with duplicated application_id values.
20,R_DUP_002,high,0.40,2,Distinct application_id keys that are duplicated.
22,R_DUP_004,high,0.40,2,Distinct SSN values that appear across differe...
8,R_APP_009,medium,7.77,39,DOB format is ambiguous NN/NN/YYYY.
17,R_APP_018,medium,3.39,17,Loan approved with less than 6 months of credi...
16,R_APP_017,medium,2.19,11,Loan approved with zero months of credit history.


In [2]:
# Export Residual Table as evidence

residual_tbl = cmp[cmp["rule_id"].isin([
    "R_APP_001","R_APP_009","R_APP_002","R_DUP_003","R_APP_003","R_DUP_001","R_APP_004","R_APP_005"
])][["rule_id","severity","post_percent","post_count","description"]].copy()

residual_tbl.to_csv("reports/evidence/residual_issues_selected.csv", index=False)